在跑模型的时候，大家可能会遇到如下报错

UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

这时只要将 torch.tensor()  改写成  torch.as_tensor() 就可以避免报错了。

In [ ]:
# 如下写法报错
 feature = torch.tensor(image, dtype=torch.float32)

# 改为
feature = torch.as_tensor(image, dtype=torch.float32)

# 1、torch.as_tensor()

new_data = torch.as_tensor(data, dtype=None,device=None)

作用：生成一个新的 tensor， 这个新生成的tensor 会根据原数据的实际情况，来决定是进行浅拷贝，还是深拷贝。当然，会优先浅拷贝，浅拷贝会共享内存，并共享 autograd 历史记录。

## 情况一：不指定新生成的 tensor 的数据类型 和 device ，那么默认新生成的 tensor 的 数据类型 和 device 同原数据的 数据类型 和 device 相同，他会帮我们进行浅拷贝，并共享内存

### 1）原数据为 array

In [ ]:
import numpy as np
import torch

a = np.array([1, 2, 3])
t = torch.as_tensor(a)
t[0] = -1

print(a)   # [-1  2  3]
print(a.dtype)   # int64
print(a.ctypes.data)   # 105553154015584

print(t)   # tensor([-1,  2,  3])
print(t.dtype)   # torch.int64
print(t.untyped_storage().data_ptr())   # 105553154015584

## 2）原数据为 tensor

In [ ]:
import torch

a = torch.tensor([1., 2., 3.], device=torch.device('cuda'))
a.grad = torch.tensor([5., 6., 7.], device=torch.device('cuda'))

t = torch.as_tensor(a)
t[0] = -1

print(a)   # tensor([-1.,  2.,  3.], device='cuda:0')
print(a.dtype)   # torch.float32

print(t)   # tensor([-1.,  2.,  3.], device='cuda:0')
print(t.dtype)   # torch.float32
print(t.grad)   # tensor([5., 6., 7.], device='cuda:0')

## 情况二：  指定新生成的 tensor 的 数据类型 或 device 和原数据不一致，则会进行 深拷贝，不再共享内存

### 1）数据类型相同，但是device不同

In [ ]:
import numpy as np
import torch

a = np.array([1, 2, 3])
t = torch.as_tensor(a, device=torch.device('cuda'))
t[0] = -1

print(a)   # [1 2 3]
print(a.dtype)   # int64
print(a.ctypes.data)   # 102285095257904

print(t)   # tensor([-1,  2,  3], device='cuda:0')
print(t.dtype)   # torch.int64
print(t.untyped_storage().data_ptr())   # 138156835864576

### 2）device相同，但数据类型不同，深拷贝，不再共享内存

In [ ]:
import numpy as np
import torch

a = np.array([1, 2, 3])
t = torch.as_tensor(a, dtype=torch.float32)
t[0] = -1

print(a)   # [1 2 3]
print(a.dtype)   # int64
print(t)   # tensor([-1.,  2.,  3.])
print(t.dtype)   # torch.float32

# 2、torch.tensor()
torch.tensor() 是深拷贝方式。

torch.tensor(data, dtype=None, device=None, requires_grad=False, pin_memory=False)

深拷贝：会拷贝 数据类型 和 device， (对于叶子 tensor，also known as “leaf tensor”）不会记录 autograd 历史

## 1）如果原数据的数据类型是 tensor，使用 torch.tensor(data) 就会报waring

In [ ]:
# 原数据类型是：tensor 会发出警告
import torch

a = torch.tensor([1, 2, 3], device=torch.device('cuda'))
t = torch.tensor(a)
t[0] = -1

print(a)  # tensor([1, 2, 3], device='cuda:0')
print(t)  # tensor([-1,  2,  3], device='cuda:0')

# 输出：
# tensor([1, 2, 3], device='cuda:0')
# tensor([-1,  2,  3], device='cuda:0')
# /opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor,
# it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True),
# rather than torch.tensor(sourceTensor).

## 2）如果原数据的数据类型是：list, tuple, NumPy ndarray, scalar, and other types，不会 waring

In [ ]:
# 原数据类型是：list, tuple, NumPy ndarray, scalar, and other types， 没警告
import torch
import numpy as np

a =  np.array([1, 2, 3])
t = torch.tensor(a)

b = [1,2,3]
t= torch.tensor(b)

c = (1,2,3)
t= torch.tensor(c)